In [2]:

import cv2
import numpy as np
import pickle
from mtcnn.mtcnn import MTCNN

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
face_lst = []

distance_threshold=0.6


[ WARN:0] global /tmp/pip-req-build-24v3eisa/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [3]:
import matplotlib.pyplot as plt
from modules.face_recog_v2 import get_encodings_from_face_array
# Load a trained KNN model (if one was passed in)

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
face_lst = []

distance_threshold=0.6

with open("models/trained_knn_model_v2.clf", 'rb') as f:
    knn_clf = pickle.load(f)

# Create a face detector instance
detector = MTCNN()

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = detector.detect_faces(rgb_small_frame)
    #     print(face_locations)
        face_locations_boxes = []
        face_names = []
        for idx, face_loc in enumerate(face_locations):
            face_locations_boxes.append(face_loc['box'])
            print(face_loc['box'])
            x1, y1, width, height = face_loc['box']
            x2, y2 = x1 + width, y1 + height

            # plot face if draw_faces is True
            cropped_face = rgb_small_frame[y1:y2, x1:x2]
            
            encoding = get_encodings_from_face_array(cropped_face)
            label = knn_clf.predict(encoding.reshape(1, -1))[0]

            face_names.append(label)

        process_this_frame = not process_this_frame


        print(face_locations_boxes)
        # Display the results
        for (top, right, bottom, left), name in zip(face_locations_boxes, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break



2021-12-06 20:23:50.554226: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/site-packages/cv2/../../lib64:
2021-12-06 20:23:50.554297: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-06 20:23:50.554337: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3f0e8b44e9b3): /proc/driver/nvidia/version does not exist
2021-12-06 20:23:50.554596: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[ WARN:0] global /tmp/pip-req-build-

error: OpenCV(4.5.4) /tmp/pip-req-build-24v3eisa/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [20]:

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

### Create Camera Widget 

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
# dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)



# # create widgets
# dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
# category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
# count_widget = ipywidgets.IntText(description='count')
# save_widget = ipywidgets.Button(description='add')

# # manually update counts at initialization
# count_widget.value = dataset.get_count(category_widget.value)

# # sets the active dataset
# def set_dataset(change):
#     global dataset
#     dataset = datasets[change['new']]
#     count_widget.value = dataset.get_count(category_widget.value)
# dataset_widget.observe(set_dataset, names='value')

# # update counts when we select a new category
# def update_counts(change):
#     count_widget.value = dataset.get_count(change['new'])
# category_widget.observe(update_counts, names='value')

# # save image for category and update counts
# def save(c):
#     dataset.save_entry(camera.value, category_widget.value)
#     count_widget.value = dataset.get_count(category_widget.value)
# save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget])
])

display(data_collection_widget)
print("data_collection_widget created")

### Live execution widget

In [ ]:
import threading
import time
import cv2
import face_recognition


state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
score_widgets = []
for category in ['seth', 'jordyn', 'claire']:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)

def live(state_widget, model, camera, prediction_widget, score_widget):
    global dataset
    while state_widget.value == 'live':
        # image = camera.value

        # Grab a single frame of video
        ret, frame = camera.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for idx, face_encoding in enumerate(face_encodings):

                # See if the face is a match for the known face(s)
                # Use the KNN model to find the best matches for the test face
                closest_distances = knn_clf.kneighbors([face_encoding], n_neighbors=1)
                
                are_matches = [closest_distances[0][0][0] <= distance_threshold]

                # Predict classes and remove classifications that aren't within the threshold
                face_lst = [knn_clf.predict([face_encoding]), face_locations[idx] if are_matches else ("unknown", face_locations[idx])]
                
                face_names.append(face_lst[0][0])
                



        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            

            category_index = output.argmax()
            prediction_widget.value = dataset.categories[category_index]
            for i, score in enumerate(list(output)):
                score_widgets[i].value = score
                
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")

In [ ]:
!pip install fr_utils
!pip install inception_blocks

In [70]:
import sys
import pandas as pd
from PIL import Image
from fr_utils import *
from inception_blocks import *
from matplotlib.pyplot import figure

%matplotlib qt

from skimage.segmentation import mark_boundaries
from lime.wrappers.scikit_image import SegmentationAlgorithm

# imports for LIME
import sklearn
import sklearn.datasets
import sklearn.ensemble

import lime
import lime.lime_tabular
from lime import lime_image
from __future__ import print_function


# loss function for image encoding comparisons
def triplet_loss(y_true, y_pred, alpha = 0.2):

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    # distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    
    # distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    
    # subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    
    # maximum of basic_loss and 0.0
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))

    return loss


# given image, returns encoding
def img_to_encoding(img1, model):
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


# given image path, returns encoding
def img_path_to_encoding(image_path, model):
    return img_to_encoding(cv2.imread(image_path, 1), model)


# given path to dir with images, fills 'database' with images
def fillDatabase(databasePath, database):
    databasePath = os.fsencode(databasePath)
    if not os.path.exists(databasePath):
        sys.exit('Invalid database path')
        
    for img in os.listdir(databasePath):
        img_path = os.path.join(databasePath, os.fsencode(img))
        database[img.decode('UTF-8')] = img_path_to_encoding(os.fsdecode(img_path), FRmodel)
        

# given list of images, gets probability of each image being 'Greg'
def get_probability(imgs):
    dists = []
    for i in range(len(imgs)):
        # runs image through FaceNet model
        encoding = img_to_encoding(imgs[i], FRmodel)
        
        # calculates distance from 'Greg'
        dist = np.linalg.norm(encoding-database["greg_anchor.jpg"])
        
        # [prob of being Greg, prob of being not Greg]
        dists.append([1-dist, dist])
    return dists


# outputs superpixel features that most significantly contribute to results
def get_explanation(img_path):
    # get image
    inputIMG = cv2.imread(img_path, 1)
    
    res = get_probability([inputIMG])[0]
    if res[0] > .3:
        print("It is Greg!")
    else:
        print("Not a Greg.")

    # get explanation from LIME
    explanation = explainer.explain_instance(inputIMG, 
                                            classifier_fn = get_probability, top_labels=2, 
                                            hide_color=0, num_samples=100, segmentation_fn=segmenter)

    # display top 5 features
    for i in range(5, 0, -1):
        temp, mask = explanation.get_image_and_mask(0, positive_only=False, num_features=i, hide_rest=False)
        plt.figure()
        plt.imshow(mark_boundaries(temp, mask))


# specify model input, compile, then load the weights
FRmodel= faceRecoModel(input_shape=(3, 96, 96))
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)


# fill database
database = {}
fillDatabase('images', database)


# init LIME explainer and segmentation function
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('slic', n_segments=50, compactness=1, sigma=1)


# see explanations
get_explanation("images/greg_positive.jpg")
get_explanation("images/maxim_positive.jpg")

ModuleNotFoundError: No module named 'fr_utils'